### Pick a subreddit. What user wrote the most comments in January of 2012? What was the user’s top three most-upvoted comments? Filter out bots or other types of automated posts.

In [47]:
subReddit = 'AskReddit'
author = ''

#### Loading data

In [48]:
# from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import from_json, col
conf = SparkConf().setAppName('FirstSpark2').setMaster('Spark')
sc = SparkContext.getOrCreate()

from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
df = sqlContext.read.json("hdfs://orion11:20001/sample_sampled_reddit/")

#### Filtering data

In [49]:
df2 = df.filter(
~(df.body.like('[deleted]'))
    & ~(df.body.isNull())
    & ~(df.author.like('[deleted]'))
    & ~(df.author.like('AutoModerator')) 
    & ~(df.author.rlike("[bB][oO][tT]"))

)

print(type(df2))
print(df2.count())
df2.show(1)

<class 'pyspark.sql.dataframe.DataFrame'>
27303462
+--------+--------------+--------------+----------------------+-----------------+--------------------+---------+----------------+-------+-----------+-------------+-----+------+------+-------+---------+-----------+----+---------+--------------+-------+------------+-----+-----+------------+--------+---------------+------------+---+------------+
|archived|        author|author_cakeday|author_flair_css_class|author_flair_text|                body|body_html|controversiality|created|created_utc|distinguished|downs|edited|gilded|     id|  link_id|mod_reports|name|parent_id|removal_reason|replies|retrieved_on|saved|score|score_hidden|stickied|      subreddit|subreddit_id|ups|user_reports|
+--------+--------------+--------------+----------------------+-----------------+--------------------+---------+----------------+-------+-----------+-------------+-----+------+------+-------+---------+-----------+----+---------+--------------+-------+--------

#### Getting the name of subreddits

In [50]:
df3 = df2.groupBy(df2.subreddit)
df3 = df3.count()

print(type(df3))
print(df3.count())
df3.orderBy('count', ascending=False).show()

<class 'pyspark.sql.dataframe.DataFrame'>
88429
+---------------+-------+
|      subreddit|  count|
+---------------+-------+
|      AskReddit|2443278|
|          funny| 545756|
|           pics| 508294|
|       politics| 441766|
|leagueoflegends| 437114|
|         gaming| 353938|
|            WTF| 305215|
|            nfl| 298155|
|  AdviceAnimals| 290879|
|      worldnews| 289444|
|         videos| 282651|
|  todayilearned| 255729|
|            nba| 243260|
|         soccer| 198731|
|           news| 197988|
|         movies| 177056|
|           IAmA| 175689|
|          DotA2| 174396|
|   pcmasterrace| 166760|
|         hockey| 166410|
+---------------+-------+
only showing top 20 rows



#### Picking subreddit - gaming

#### Filtering the data to have comments for subreddit - "gaming", in the month - January and year - 2012

In [51]:
from pyspark.sql.functions import year, month, dayofmonth, from_unixtime
from pyspark.sql.types import DateType

df4 = df2.where(df2.subreddit.like(subReddit)).where(year(from_unixtime("created_utc").cast(DateType()))==2012).where(month(from_unixtime("created_utc").cast(DateType()))== 1)

print(type(df4))
print(df4.count())
df4.show(1)


<class 'pyspark.sql.dataframe.DataFrame'>
16938
+--------+------------+--------------+----------------------+-----------------+--------------------+---------+----------------+-------+-----------+-------------+-----+------+------+-------+--------+-----------+----------+---------+--------------+-------+------------+-----+-----+------------+--------+---------+------------+---+------------+
|archived|      author|author_cakeday|author_flair_css_class|author_flair_text|                body|body_html|controversiality|created|created_utc|distinguished|downs|edited|gilded|     id| link_id|mod_reports|      name|parent_id|removal_reason|replies|retrieved_on|saved|score|score_hidden|stickied|subreddit|subreddit_id|ups|user_reports|
+--------+------------+--------------+----------------------+-----------------+--------------------+---------+----------------+-------+-----------+-------------+-----+------+------+-------+--------+-----------+----------+---------+--------------+-------+------------+-

#### For the above subreddit fetching the author who has made the most number of comments in January 2012

In [52]:
groupedByAuthor = df4.groupBy("author").count().orderBy('count', ascending=False)
groupedByAuthor.show(5)


+---------------+-----+
|         author|count|
+---------------+-----+
|NinjaDiscoJesus|   35|
|       iam4real|   24|
|andrewsmith1986|   23|
|        jhudsui|   22|
|         Lots42|   21|
+---------------+-----+
only showing top 5 rows



#### Selecting top author

In [53]:
author = groupedByAuthor.select('author').head(1)
author =(author[0]).author
print(type(author))
print(author)

<class 'str'>
NinjaDiscoJesus


#### odering author comments by ups (decending)

In [54]:
df5 = df4.where(df4.subreddit.like(subReddit)).where(df4.author == author).orderBy('ups', ascending=False)
df5.select(df5.id, df5.subreddit, df5.author, df5.body, df5.ups).show(3, False)

+-------+---------+---------------+----------------------------------------------------------------------------------------------------------+---+
|id     |subreddit|author         |body                                                                                                      |ups|
+-------+---------+---------------+----------------------------------------------------------------------------------------------------------+---+
|c3mf9mb|AskReddit|NinjaDiscoJesus|I dunno... shouldn't both be kept quiet until the verdict.. then only the victim should have protection.. |57 |
|c3g3q8s|AskReddit|NinjaDiscoJesus|You don't drink anyway near an amount that would be considered 'a lot'                                    |32 |
|c3i87tn|AskReddit|NinjaDiscoJesus|nope.. neither did the mayans I believe                                                                   |8  |
+-------+---------+---------------+-----------------------------------------------------------------------------------